In [6]:
import pandas as pd

In [7]:
contributions = pd.read_csv('../data_processing/individual_contributions.csv', sep='|', header=None,
                            usecols=[0, 3, 10, 13, 14, 20],
                            names=['CMTE_ID',
                                  'TRANSACTION_PGI',
                                  'ZIP_CODE',
                                  'TRANSACTION_DT',
                                  'TRANSACTION_AMT',
                                  'SUB_ID'],
                            dtype={"CMTE_ID": str, 
                                   "TRANSACTION_PGI": str,
                                   "ZIP_CODE": str,
                                  'TRANSACTION_DT': str,
                                   "TRANSACTION_AMT": float, 
                                   "SUB_ID": str
                                  }
                           )

In [8]:
contributions['zipcode_5'] = contributions['ZIP_CODE'].str[0:5]
contributions['ZIP_CODE'] = pd.to_numeric(contributions['ZIP_CODE'], errors='coerce')

In [9]:
contributions.dropna(subset=['CMTE_ID', 'ZIP_CODE','TRANSACTION_AMT','SUB_ID','zipcode_5'],inplace=True)
contributions = contributions.loc[contributions['TRANSACTION_AMT'] > 0]

In [10]:
candidate_committee = pd.read_csv('../data_processing/candidate_committee.csv', sep='|')
candidate_committee = candidate_committee[['CAND_ID', 'CMTE_ID']]

In [11]:
candidates = pd.read_csv('../data_processing/candidates.csv', sep='|', names=['CAND_ID', 'CAND_PARTY'], usecols=[0, 2],).iloc[1:, :]
#candidates = candidates[['CAND_ID', 'CMTE_ID']]
candidates.head()
contributions = contributions.merge(candidate_committee.merge(candidates, how='inner'))

In [29]:
grouped_party = contributions.groupby(['zipcode_5', 'CAND_PARTY'])
party_summary_df = pd. ()
party_summary_df['donations_sum'] = grouped_party['TRANSACTION_AMT'].sum()
party_summary_df['donations_median'] = grouped_party['TRANSACTION_AMT'].median()
party_summary_df['donations_count'] = grouped_party['SUB_ID'].count()
party_summary_df.head()

donations_sum  donations_median  donations_count
zipcode_5 CAND_PARTY                                                  
 3102     DEM                 400.0             100.0                4
00000     DEM             1187085.0              50.0             7566
          DFL                 100.0             100.0                1
          REP                9885.0              42.5               22
00001     DEM                 550.0             275.0                2

In [39]:
new_party_summary_df = party_summary_df.reset_index()
new_party_summary_df = new_party_summary_df.pivot(index='zipcode_5', columns='CAND_PARTY').fillna(0)
new_party_summary_df.head()

donations_sum                                                       \
CAND_PARTY           AMP  CON        DEM    DFL  GRE  IDP  IND  LIB  LPF  NNE   
zipcode_5                                                                       
 3102                0.0  0.0      400.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0   
00000                0.0  0.0  1187085.0  100.0  0.0  0.0  0.0  0.0  0.0  0.0   
00001                0.0  0.0      550.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0   
00003                0.0  0.0      600.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0   
00006                0.0  0.0      275.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0   

           ...  donations_count                                                
CAND_PARTY ...              NON  NPA  NPP  OTH  PPT   REP   UN  UNK    W  WFP  
zipcode_5  ...                                                                 
 3102      ...              0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  
00000      ...              0.0  0.0  0.0  0.0  0.0  22.0  0.0  0.0  0.0  0.0  
00001      ...              0.0  0.0  0.0  0.0  0.0   1.0  0.0  0.0  0.0  0.0  
00003      ...              0.0  0.0  0.0  0.0  0.0   1.0  0.0  0.0  0.0  0.0  
00006      ...              0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  

[5 rows x 60 columns]

In [17]:
census_df = pd.read_csv('census_data_2016.csv', sep=',`')

C:\Users\alexa\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,76,78,80,82,84,86,87,88,89,90,91,92,93,94,95,96,97,101,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,150,151,152,153,154,155,156,157,158,159,160,161,162,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
census_df.columns = census_df.iloc[0]
census_df.drop(0, axis=0, inplace=True)
census_df.head()

,Id,Id2,Geography,Estimate; Total:,Estimate; Total: - White alone,Estimate; Total: - Black or African American alone,Estimate; Total: - American Indian and Alaska Native alone,Estimate; Total: - Asian alone,Estimate; Total: - Native Hawaiian and Other Pacific Islander alone,Estimate; Total: - Some other race alone,...,Total; Estimate; POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL - Less than high school graduate,Total; Estimate; POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL - High school graduate (includes equivalency),Total; Estimate; POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL - Some college or associate's degree,Total; Estimate; POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL - Bachelor's degree or higher,Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings,Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Less than high school graduate,Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - High school graduate (includes equivalency),Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Some college or associate's degree,Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Bachelor's degree,Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Graduate or professional degree
1,8600000US00601,601,ZCTA5 00601,17800,14436,108,21,0,0,3046,...,(X),(X),(X),(X),16533,11522,15606,13561,22859,36771
2,8600000US00602,602,ZCTA5 00602,39716,22941,968,0,74,0,1544,...,(X),(X),(X),(X),13682,8018,12037,13060,21037,26118
3,8600000US00603,603,ZCTA5 00603,51565,35176,2149,106,638,11,6610,...,(X),(X),(X),(X),16022,9848,11884,15023,24438,32807
4,8600000US00606,606,ZCTA5 00606,6320,3739,175,0,0,0,2246,...,(X),(X),(X),(X),10861,6116,12798,13690,14466,-
5,8600000US00610,610,ZCTA5 00610,27976,16259,1018,0,33,1,6871,...,(X),(X),(X),(X),15167,8367,14411,14419,21234,33125


In [23]:
census_df['zipcode_5'] = census_df['Id'].str[9:14]
new_census_df = census_df.iloc[:, 3:]
new_census_df.set_index('zipcode_5', inplace=True)
new_census_df.head()

,Estimate; Total:,Estimate; Total: - White alone,Estimate; Total: - Black or African American alone,Estimate; Total: - American Indian and Alaska Native alone,Estimate; Total: - Asian alone,Estimate; Total: - Native Hawaiian and Other Pacific Islander alone,Estimate; Total: - Some other race alone,Estimate; Total: - Two or more races:,Estimate; Total: - Two or more races: - Two races including Some other race,"Estimate; Total: - Two or more races: - Two races excluding Some other race, and three or more races",...,Total; Estimate; POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL - Less than high school graduate,Total; Estimate; POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL - High school graduate (includes equivalency),Total; Estimate; POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL - Some college or associate's degree,Total; Estimate; POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL - Bachelor's degree or higher,Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings,Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Less than high school graduate,Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - High school graduate (includes equivalency),Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Some college or associate's degree,Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Bachelor's degree,Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Graduate or professional degree
zipcode_5,,,,,,,,,,,,,,,,,,,,,
00601,17800,14436,108,21,0,0,3046,189,26,163,...,(X),(X),(X),(X),16533,11522,15606,13561,22859,36771
00602,39716,22941,968,0,74,0,1544,14189,164,14025,...,(X),(X),(X),(X),13682,8018,12037,13060,21037,26118
00603,51565,35176,2149,106,638,11,6610,6875,333,6542,...,(X),(X),(X),(X),16022,9848,11884,15023,24438,32807
00606,6320,3739,175,0,0,0,2246,160,77,83,...,(X),(X),(X),(X),10861,6116,12798,13690,14466,-
00610,27976,16259,1018,0,33,1,6871,3794,103,3691,...,(X),(X),(X),(X),15167,8367,14411,14419,21234,33125


In [24]:
for column in new_census_df.columns[:211]:
    new_census_df[column] = pd.to_numeric(new_census_df[column], errors='coerce')
new_census_df.dropna(inplace=True, axis=1)
new_census_df.head()

,Estimate; Total:,Estimate; Total: - White alone,Estimate; Total: - Black or African American alone,Estimate; Total: - American Indian and Alaska Native alone,Estimate; Total: - Asian alone,Estimate; Total: - Native Hawaiian and Other Pacific Islander alone,Estimate; Total: - Some other race alone,Estimate; Total: - Two or more races:,Estimate; Total: - Two or more races: - Two races including Some other race,"Estimate; Total: - Two or more races: - Two races excluding Some other race, and three or more races",...,Total; Estimate; Native Hawaiian and Other Pacific Islander alone - Bachelor's degree or higher,Total; Estimate; Some other race alone,Total; Estimate; Some other race alone - High school graduate or higher,Total; Estimate; Some other race alone - Bachelor's degree or higher,Total; Estimate; Two or more races,Total; Estimate; Two or more races - High school graduate or higher,Total; Estimate; Two or more races - Bachelor's degree or higher,Total; Estimate; Hispanic or Latino Origin,Total; Estimate; Hispanic or Latino Origin - High school graduate or higher,Total; Estimate; Hispanic or Latino Origin - Bachelor's degree or higher
zipcode_5,,,,,,,,,,,,,,,,,,,,,
00601,17800,14436,108,21,0,0,3046,189,26,163,...,0,1924,1275,436,114,92,11,11863,7488,2217
00602,39716,22941,968,0,74,0,1544,14189,164,14025,...,0,849,653,218,9677,6399,2131,25503,16080,5073
00603,51565,35176,2149,106,638,11,6610,6875,333,6542,...,0,4474,3221,1100,4725,3033,1006,34582,23936,7604
00606,6320,3739,175,0,0,0,2246,160,77,83,...,0,1479,770,128,131,44,7,4370,2382,426
00610,27976,16259,1018,0,33,1,6871,3794,103,3691,...,0,4327,2950,560,2446,1705,326,18758,12590,2901


In [58]:
new_summary_df = pd.DataFrame()
new_summary_df['donations_sum_DEM'] = new_party_summary_df['donations_sum', 'DEM']
new_summary_df['donations_median_DEM'] = new_party_summary_df['donations_median', 'DEM']
new_summary_df['donations_sum_REP'] = new_party_summary_df['donations_sum', 'REP']
new_summary_df['donations_median_REP'] = new_party_summary_df['donations_median', 'REP']
new_summary_df.head()

,donations_sum_DEM,donations_median_DEM,donations_sum_REP,donations_median_REP
zipcode_5,,,,
3102,400,100.0,0.0,0.0
00000,1.18708e+06,50.0,9885.0,42.5
00001,550,275.0,2700.0,2700.0
00003,600,250.0,1000.0,1000.0
00006,275,75.0,0.0,0.0


In [59]:
combined_summary_df = new_census_df.join(new_summary_df, how='inner')
combined_summary_df.head()

,Estimate; Total:,Estimate; Total: - White alone,Estimate; Total: - Black or African American alone,Estimate; Total: - American Indian and Alaska Native alone,Estimate; Total: - Asian alone,Estimate; Total: - Native Hawaiian and Other Pacific Islander alone,Estimate; Total: - Some other race alone,Estimate; Total: - Two or more races:,Estimate; Total: - Two or more races: - Two races including Some other race,"Estimate; Total: - Two or more races: - Two races excluding Some other race, and three or more races",...,Total; Estimate; Two or more races,Total; Estimate; Two or more races - High school graduate or higher,Total; Estimate; Two or more races - Bachelor's degree or higher,Total; Estimate; Hispanic or Latino Origin,Total; Estimate; Hispanic or Latino Origin - High school graduate or higher,Total; Estimate; Hispanic or Latino Origin - Bachelor's degree or higher,donations_sum_DEM,donations_median_DEM,donations_sum_REP,donations_median_REP
zipcode_5,,,,,,,,,,,,,,,,,,,,,
00602,39716,22941,968,0,74,0,1544,14189,164,14025,...,9677,6399,2131,25503,16080,5073,1891,50.0,0.0,0.0
00603,51565,35176,2149,106,638,11,6610,6875,333,6542,...,4725,3033,1006,34582,23936,7604,3923,15.0,0.0,0.0
00606,6320,3739,175,0,0,0,2246,160,77,83,...,131,44,7,4370,2382,426,0,0.0,500.0,500.0
00610,27976,16259,1018,0,33,1,6871,3794,103,3691,...,2446,1705,326,18758,12590,2901,1157,5.0,0.0,0.0
00612,63884,52902,3346,252,678,0,5209,1497,523,974,...,1011,871,205,43307,32610,10094,2410,50.0,0.0,0.0


In [60]:
new_census_df.columns

Index(['Estimate; Total:', 'Estimate; Total: - White alone',
       'Estimate; Total: - Black or African American alone',
       'Estimate; Total: - American Indian and Alaska Native alone',
       'Estimate; Total: - Asian alone',
       'Estimate; Total: - Native Hawaiian and Other Pacific Islander alone',
       'Estimate; Total: - Some other race alone',
       'Estimate; Total: - Two or more races:',
       'Estimate; Total: - Two or more races: - Two races including Some other race',
       'Estimate; Total: - Two or more races: - Two races excluding Some other race, and three or more races',
       ...
       'Total; Estimate; Native Hawaiian and Other Pacific Islander alone - Bachelor's degree or higher',
       'Total; Estimate; Some other race alone',
       'Total; Estimate; Some other race alone - High school graduate or higher',
       'Total; Estimate; Some other race alone - Bachelor's degree or higher',
       'Total; Estimate; Two or more races',
       'Total; Estimat

In [61]:
combined_summary_df['donations_sum_DEM_quartile'] = pd.qcut(combined_summary_df['donations_sum_DEM'], q=[0, .25, .5, .75, 1.], labels=[4, 3, 2, 1])
combined_summary_df['donations_median_DEM_quartile'] = pd.qcut(combined_summary_df['donations_median_DEM'], q=[0, .25, .5, .75, 1.], labels=[4, 3, 2, 1])
combined_summary_df['donations_sum_REP_quartile'] = pd.qcut(combined_summary_df['donations_sum_REP'], q=[0, .25, .5, .75, 1.], labels=[4, 3, 2, 1])
combined_summary_df['donations_median_REP_quartile'] = pd.qcut(combined_summary_df['donations_median_REP'], q=[0, .25, .5, .75, 1.], labels=[4, 3, 2, 1])

In [62]:
combined_summary_df.sort_values(by=['donations_sum_DEM'])

,Estimate; Total:,Estimate; Total: - White alone,Estimate; Total: - Black or African American alone,Estimate; Total: - American Indian and Alaska Native alone,Estimate; Total: - Asian alone,Estimate; Total: - Native Hawaiian and Other Pacific Islander alone,Estimate; Total: - Some other race alone,Estimate; Total: - Two or more races:,Estimate; Total: - Two or more races: - Two races including Some other race,"Estimate; Total: - Two or more races: - Two races excluding Some other race, and three or more races",...,Total; Estimate; Hispanic or Latino Origin - High school graduate or higher,Total; Estimate; Hispanic or Latino Origin - Bachelor's degree or higher,donations_sum_DEM,donations_median_DEM,donations_sum_REP,donations_median_REP,donations_sum_DEM_quartile,donations_median_DEM_quartile,donations_sum_REP_quartile,donations_median_REP_quartile
zipcode_5,,,,,,,,,,,,,,,,,,,,,
49073,4784,4628,13,5,4,0,36,98,11,87,...,39,7,0,0.0,14585.0,50.0,4,4,2,4
64101,309,159,140,1,0,0,0,9,0,9,...,9,0,0,0.0,0.0,0.0,4,4,4,4
47018,4495,4383,21,0,21,55,8,7,0,7,...,8,0,0,0.0,1240.0,35.0,4,4,3,4
47017,327,327,0,0,0,0,0,0,0,0,...,0,0,0,0.0,200.0,100.0,4,4,4,3
64125,1303,852,99,16,0,0,334,2,2,0,...,90,5,0,0.0,755.0,62.5,4,4,3,3
47016,1136,1133,0,0,0,0,0,3,0,3,...,0,0,0,0.0,401.0,30.0,4,4,4,4
46991,1707,1701,0,4,0,0,0,2,0,2,...,0,0,0,0.0,350.0,175.0,4,4,4,2
64147,639,229,364,0,4,0,3,39,8,31,...,0,0,0,0.0,500.0,500.0,4,4,4,1
64096,1118,989,58,0,0,0,18,53,0,53,...,31,2,0,0.0,99.0,99.0,4,4,4,3


In [63]:
combined_summary_df.to_csv('cleaned_combined_zipcodes_new.csv', index=False)

In [41]:
combined_summary_df = pd.read_csv('cleaned_combined_zipcodes.csv')

In [ ]:
# Create Models 

In [66]:
from numpy.random import seed
#seed(1)
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

In [109]:
group_type = 'median'
party = 'REP'
combined_summary_df.dropna(inplace=True)
X = combined_summary_df[combined_summary_df.columns[:163]]
y = combined_summary_df[f"donations_{group_type}_{party}_quartile"]
print(X.shape, y.shape)

(27463, 163) (27463,)


In [110]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)

In [111]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [112]:
# The output layer has 5 columns that are one-hot encoded
y_train.shape

(20597, 5)

In [113]:
y_test

array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [114]:
# Create an empty sequential model
model = Sequential()

In [115]:
# Add the first layer where the input dimensions are the 561 columns of the training data
model.add(Dense(200, activation='relu', input_dim=X_train.shape[1]))

In [116]:
# Add a second hidden layer
model.add(Dense(200, activation='relu'))

In [117]:
# Add output layer
model.add(Dense(y_train.shape[1], activation="softmax"))

In [118]:
# Compile the model using categorical_crossentropy for the loss function, the adam optimizer,
# and add accuracy to the training metrics
model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])

In [119]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [120]:
# Use the training data to fit (train) the model
model.fit(
    X_train_scaled,
    y_train,
    epochs=20,
    shuffle=True,
    verbose=2
)

Epoch 1/20
 - 2s - loss: 1.2913 - acc: 0.4156
Epoch 2/20
 - 1s - loss: 1.2515 - acc: 0.4296
Epoch 3/20
 - 1s - loss: 1.2402 - acc: 0.4354
Epoch 4/20
 - 2s - loss: 1.2360 - acc: 0.4367
Epoch 5/20
 - 2s - loss: 1.2288 - acc: 0.4420
Epoch 6/20
 - 1s - loss: 1.2239 - acc: 0.4452
Epoch 7/20
 - 1s - loss: 1.2184 - acc: 0.4488
Epoch 8/20
 - 2s - loss: 1.2188 - acc: 0.4452
Epoch 9/20
 - 1s - loss: 1.2109 - acc: 0.4492
Epoch 10/20
 - 1s - loss: 1.2058 - acc: 0.4547
Epoch 11/20
 - 1s - loss: 1.2055 - acc: 0.4556
Epoch 12/20
 - 2s - loss: 1.1992 - acc: 0.4597
Epoch 13/20
 - 1s - loss: 1.1949 - acc: 0.4621
Epoch 14/20
 - 1s - loss: 1.1884 - acc: 0.4689
Epoch 15/20
 - 2s - loss: 1.1842 - acc: 0.4710
Epoch 16/20
 - 1s - loss: 1.1793 - acc: 0.4699
Epoch 17/20
 - 1s - loss: 1.1706 - acc: 0.4754
Epoch 18/20
 - 1s - loss: 1.1616 - acc: 0.4829
Epoch 19/20
 - 2s - loss: 1.1530 - acc: 0.4861
Epoch 20/20
 - 1s - loss: 1.1445 - acc: 0.4914


In [123]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

 - 0s - loss: 1.2942 - acc: 0.4141
Loss: 1.2942126557391551, Accuracy: 0.4140693247318268


In [124]:
model.save(f'donations_{group_type}_{party}_model.h5')

In [131]:
# Load models

In [136]:
from tensorflow.keras.models import load_model
group_type = 'median'
party = 'DEM'
model = load_model(f'donations_{group_type}_{party}_model.h5')

In [137]:
combined_summary_df[f'donations_{group_type}_{party}_predicted'] = model.predict_classes(X_scaler.transform(X))
combined_summary_df.head(50)

,Estimate; Total:,Estimate; Total: - White alone,Estimate; Total: - Black or African American alone,Estimate; Total: - American Indian and Alaska Native alone,Estimate; Total: - Asian alone,Estimate; Total: - Native Hawaiian and Other Pacific Islander alone,Estimate; Total: - Some other race alone,Estimate; Total: - Two or more races:,Estimate; Total: - Two or more races: - Two races including Some other race,"Estimate; Total: - Two or more races: - Two races excluding Some other race, and three or more races",...,donations_sum_REP,donations_median_REP,donations_sum_DEM_quartile,donations_median_DEM_quartile,donations_sum_REP_quartile,donations_median_REP_quartile,donations_median_REP_predicted,donations_sum_DEM_predicted,donations_sum_REP_predicted,donations_median_DEM_predicted
zipcode_5,,,,,,,,,,,,,,,,,,,,,
00602,39716,22941,968,0,74,0,1544,14189,164,14025,...,0.0,0.0,2,2,4,4,4,1,4,1
00603,51565,35176,2149,106,638,11,6610,6875,333,6542,...,0.0,0.0,2,4,4,4,4,2,4,3
00606,6320,3739,175,0,0,0,2246,160,77,83,...,500.0,500.0,4,4,4,1,1,4,4,4
00610,27976,16259,1018,0,33,1,6871,3794,103,3691,...,0.0,0.0,3,4,4,4,4,4,4,4
00612,63884,52902,3346,252,678,0,5209,1497,523,974,...,0.0,0.0,2,2,4,4,4,2,4,3
00622,7385,4530,95,0,0,0,2750,10,0,10,...,0.0,0.0,4,2,4,4,4,4,4,3
00623,42711,25586,1457,38,11,0,15212,407,42,365,...,0.0,0.0,2,4,4,4,4,2,4,1
00624,23945,19874,749,37,46,0,2671,568,110,458,...,0.0,0.0,4,4,4,4,4,4,4,4
00627,33481,26674,716,38,0,0,5734,319,154,165,...,0.0,0.0,3,1,4,4,4,3,4,1


In [140]:
combined_summary_df[combined_summary_df.columns[167:]].to_csv('zipcodes_quartiles.csv')